<a href="https://colab.research.google.com/github/Muvazana/GeneticAlgoritm_1/blob/master/Genetic%20Algoritm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Moch. Nauval Rizaldi Nasril

Structure Model:

- Population Model   : Generational Model
- Parent Selection   : Tournament Selection
- Xover Model        : One Point for Binary
- Mutation Model     : Mutation for Binary
- Selection Survivor : Elitism

Structure Program: 

- POPULATION    is GENERATION
- POPULATION    contain List of INDIVIDUAL
- INDIVIDUAL    contain Fitness and KROMOSOM
- KROMOSOM      contain List of GEN
- GEN           Type {Binary, Integer, Real, Permutation}

In [1]:
!pip install beautifultable # Installation for BeautifulTable Library
import numpy as np
from beautifultable import BeautifulTable

interval_x_y = ([-1, 2], [-1, 1])

In [2]:
def myFunc(x, y):
    # this fucntion will return the calculate from your function you have
    return (x ** 2 * np.sin(y ** 2)) + (x + y)

In [3]:
def fitness(f):
    # if we want to get the maxima we use h = f
    # if we wamt to get the minimum output we use 1/(f+a) the var you can assign 0.01 to prevent 0
    return f

In [4]:
def createKromosom():
    # this function create the random Kromosom where the GEN_TYPE is Binary
    # you can change the type of gen to integer where the interval from 0 to 9
    # the kromosom size is size of gen in one kromosom
    gen_type = [0, 1]  # Binary [0,1] / Integer [0,9]
    kromosom_size = 20
    return np.random.choice(gen_type, kromosom_size)

In [5]:
def decodeForBinary(interval_x, interval_y, kromosom):
    # this function will decode and return your kromosom to x and y, where the GEN_TYPE is Binary
    # the interval_x for interval x
    # the interval_y for interval y
    # formula => x = r_b + ((r_a - r_b)/sum(i=1 to N; 2^-i)) * sum(i=1 to N; g_i^-i)
    # for r_a (upper limir) and r_b (bottom_limit)
    krom_x = kromosom[:len(kromosom)//2]
    krom_y = kromosom[len(kromosom)//2:]

    pembagi = 0    # sum(i=1 to N; 2^-i)
    pengali_x = 0  # sum(i=1 to N; g_i^-i) for x
    pengali_y = 0  # sum(i=1 to N; g_i^-i) for y

    for i in range(len(kromosom)//2):
        pembagi += 2 ** -(i+1)
        pengali_x += krom_x[i] * 2 ** -(i+1)
        pengali_y += krom_y[i] * 2 ** -(i+1)

    decodeFormula = lambda interval, pengali: interval[0] + ((interval[1] - interval[0]) / pembagi) * pengali # penjelasannya mengikuti formula (ps-> lambda: adalah fungsi yang dijadikan satu line)

    return decodeFormula(interval_x, pengali_x), decodeFormula(interval_y, pengali_y)

In [6]:
def createIndividual(kromosom):
    x, y = decodeForBinary(interval_x= interval_x_y[0], interval_y= interval_x_y[1], kromosom=kromosom)
    fit = fitness(myFunc(x, y))

    return {'fitness': fit, 'kromosom': kromosom} # dictionary

In [7]:
def tournamentSelection(population):
    # mengambil secara acak sejumlah (tournament_size) dan di tarungkan
    # pengambilan pemenang berdasarkan fitness tertinggi
    tournament_size = 2
    rand_tournament = lambda: np.random.choice(population, tournament_size, replace=False) # proses pengambilan peserta tournament (ps-> lambda: adalah fungsi yang dijadikan satu line)
    parent_1 = max(rand_tournament(), key=lambda x: x['fitness']) # proses pengambilan pemenang berdasarkan fitness (ps-> "key=" adalah parameter yang akan dijadikan acuan untuk perhitungan pemenang)
    parent_2 = max(rand_tournament(), key=lambda x: x['fitness']) # proses pengambilan pemenang berdasarkan fitness (ps-> "key=" adalah parameter yang akan dijadikan acuan untuk perhitungan pemenang)
    return parent_1['kromosom'], parent_2['kromosom']

In [8]:
def xoverBinerOnePoint(krom_parent_1, krom_parent_2, prob):
    child1, child2 = [], []
    randomProb = np.random.random()
    if randomProb < prob:
        randomPoint = np.random.randint(0, len(krom_parent_1))
        child1[:randomPoint], child1[randomPoint:] = krom_parent_1[:randomPoint], krom_parent_2[randomPoint:]
        child2[:randomPoint], child2[randomPoint:] = krom_parent_2[:randomPoint], krom_parent_1[randomPoint:]
        return child1, child2

    return krom_parent_1, krom_parent_2

In [9]:
def mutasiBiner(kromosom, prob):
    mutated_krom = []
    mutasi = False
    for i in range(len(kromosom)):
        random_prob = np.random.random()
        if random_prob < prob:
            mutasi = True
            mutated_krom.append(0 if kromosom[i] == 1 else 1)
        else:
            mutated_krom.append(kromosom[i])

    return mutated_krom, mutasi

In [10]:
def createPopulation(populationSize):
    population = []
    for _ in range(populationSize):
        population.append(createIndividual(list(createKromosom())))

    return population

In [11]:
def showPopulation(population):
    table = BeautifulTable(maxwidth=150)
    table.set_style(BeautifulTable.STYLE_BOX_DOUBLED)
    table.columns.header = ["Fitness", "x", "y", "Kromosom"]
    tabRowHeader = []
    for i in range(len(population)):
        tabRowHeader.append(str(i+1))
        x, y = decodeForBinary(interval_x=interval_x_y[0], interval_y=interval_x_y[1], kromosom=population[i]['kromosom'])
        table.rows.append([population[i]['fitness'], x, y, population[i]['kromosom']])

    table.rows.header = tabRowHeader
    print(table)

In [12]:
def getBestKromosom(population):
    return max([individual for individual in population], key=lambda x: x['fitness'])

def getMinKromosom(population):
    return min([individual for individual in population], key=lambda x: x['fitness'])

In [ ]:
if __name__ == '__main__':
    population_size = 10  # the individu will generate in one population
    generation = 30  # the generation for generate solution (Bisa dirubah sesuka hati)
    xover_prob = 0.9
    mutation_prob = 0.01

    population_list = createPopulation(population_size)

    for gene in range(generation):
        print(f"*--------------------------------Generation {gene}--------------------------------*")
        showPopulation(population_list)
        best_individual = getBestKromosom(population_list)
        print(f"----------Best Kromosom for Generation {gene}----------")
        print(f"Fitness\t\t: {best_individual['fitness']:.3f}")
        print(f"Kromosom\t: {best_individual['kromosom']}")

        print("\nCreating new Generation...")
        # Masuk proses Mating Pool / proses pembuatan populasi baru
        new_population_list = []
        for i in range(len(population_list)//2): 
            print(f"---------->Mating Pool {i + 1}")
            krom_parent_1, krom_parent_2 = tournamentSelection(population_list)
            krom_child_1, krom_child_2 = xoverBinerOnePoint(krom_parent_1, krom_parent_2, prob=xover_prob)
            print(f"Parent 1 {krom_parent_1}\nParent 2 {krom_parent_2}")
            print(f"\tChild 1 {krom_child_1}\n\tChild 2 {krom_child_2}")
            krom_child_1, mutasi1 = mutasiBiner(krom_child_1, mutation_prob)
            krom_child_2, mutasi2 = mutasiBiner(krom_child_2, mutation_prob)
            new_population_list.append(createIndividual(krom_child_1))
            new_population_list.append(createIndividual(krom_child_2))
            print(f"\t----------Mutasi----------")
            print(f"\tChild 1 Mutation : {krom_child_1} ({'Termutasi' if mutasi1 else 'Secure'})\n\tChild 2 Mutation : {krom_child_2} ({'Termutasi' if mutasi2 else 'Secure'})")

        # Elitism Process
        worst_individual = getMinKromosom(new_population_list)
        new_population_list.remove(worst_individual)
        new_population_list.append(best_individual)
        population_list = new_population_list

    # karena dalam proses mating pool nggak dikeluarin ditampilin last generationnya jadi ditampilin di luar loopnya
    print(f"*--------------------------------Generation {gene + 1}--------------------------------*")
    showPopulation(population_list)
    best_individual = getBestKromosom(population_list)
    print(f"----------Best Kromosom for Generation {gene + 1}----------")
    print(f"Fitness\t\t: {best_individual['fitness']:.3f}")
    print(f"Kromosom\t: {best_individual['kromosom']}")
    x, y = decodeForBinary(interval_x=interval_x_y[0], interval_y=interval_x_y[1], kromosom=best_individual['kromosom'])
    print(f"x : {x}\ny : {y}")